# Pods

Podman has the hability to run containers in Pods, like in Kubernetes.

Pod is a concept in which a group of one or more containers working together for a common purpose and sharing the same namespaces and cgroups (resource constraints).
This way it is also possible to group containers's network space and volume mounts, making it easier for the containers to interact and work as if they were a single unit.


## Running Pods

To run containers in pods requires to plan a little bit, as the pod needs to be created up-front, specifying the volumes and ports that will be required.
Afterwards the containers will be attached to this pod.

We will run the following example to demonstrate how it works, start a simple http server in one pod, while in another we will refresh the index.html using a volume: 2 concerns (2 containers and 1 pod):

1. Create a pod
1. Create the web server container and attach it to the pod
1. Create the sidecar container that will update the index.html



### Create the Pod

After creating the path for the volume mount we will:

1. Specify the port binding (8080:80)
1. Specify a volume mount to `/usr/local/apache2/htdocs` with the `z` parameter. `z` specifies that the volume mount should happen in each container (shared).

In [2]:
podman volume create webserver-data

: 1

In [11]:
podman pod create -p 8080:80 --name dynamic-webserver --volume  webserver-data:/usr/local/apache2/htdocs:z

d73f9bc69c6c33ff0b8bcfef53b85ad8300393e2a179621520553bfc004517c2


: 1

In [12]:
podman pod ps

POD ID        NAME               STATUS      CREATED       INFRA ID      # OF CONTAINERS
d73f9bc69c6c  dynamic-webserver  Created     1 second ago  6ff3524a368e  1


: 1

### Inspecting the pod (right after creation)

Although we just created a pod without explicitly attaching containers to it, there is already a running container in this pod: the `pause` container from the Kubernetes project.
It is in charge of maintaining the namespaces and networks from the whole pod.

Let's check this out:

In [13]:
podman ps --pod

CONTAINER ID  IMAGE                                    COMMAND     CREATED         STATUS             PORTS                 NAMES               POD ID        PODNAME
4b9f9d36290a  localhost/podman-pause:4.1.1-1655914710              18 minutes ago  Up 18 minutes ago  0.0.0.0:8080->80/tcp  eb28e60a4641-infra  eb28e60a4641  dynamic-webserver


: 1

In [14]:
podman inspect $(podman ps | awk '/pause/{ print $NF }' | grep infra) --format '{{.Mounts}}'

[{volume webserver-data /var/lib/containers/storage/volumes/webserver-data/_data /usr/local/apache2/htdocs local z [nosuid nodev rbind] true rprivate}]


: 1

In [15]:
podman inspect $(podman ps | awk '/pause/{ print $NF }' | grep infra) --format '{{.HostConfig.PortBindings}}'

map[80/tcp:[{ 8080}]]


: 1

### Attach the containers

1. First we create the httpd container, which simply serves the index.html file from the htdocs folder
1. Then we start a sidecar container that will update the index adding timestamps. This is a silly test whose purpose is to demonstrate the volume share.

Note that we are not supplying ports nor volumes to the single containers, they inherit those from the pod.

In [16]:
podman run -d --name apache-httpd --pod dynamic-webserver httpd:2.4

37303e17edb06c20421084c116fd097e35163bc9c9c7eed381985aa6a395c540


: 1

In [17]:
curl -s localhost:8080

<html><body><h1>It works!</h1></body></html>
Index Updated @Mon Jul 25 08:08:12 UTC 2022
Index Updated @Mon Jul 25 08:08:22 UTC 2022
Index Updated @Mon Jul 25 08:08:32 UTC 2022
Index Updated @Mon Jul 25 08:08:42 UTC 2022
Index Updated @Mon Jul 25 08:08:52 UTC 2022
Index Updated @Mon Jul 25 08:09:02 UTC 2022
Index Updated @Mon Jul 25 08:09:12 UTC 2022
Index Updated @Mon Jul 25 08:09:22 UTC 2022
Index Updated @Mon Jul 25 08:09:32 UTC 2022
Index Updated @Mon Jul 25 08:09:42 UTC 2022
Index Updated @Mon Jul 25 08:09:52 UTC 2022
Index Updated @Mon Jul 25 08:10:02 UTC 2022
Index Updated @Mon Jul 25 08:10:12 UTC 2022
Index Updated @Mon Jul 25 08:10:22 UTC 2022
Index Updated @Mon Jul 25 08:10:32 UTC 2022
Index Updated @Mon Jul 25 08:10:42 UTC 2022
Index Updated @Mon Jul 25 08:10:52 UTC 2022
Index Updated @Mon Jul 25 08:11:02 UTC 2022
Index Updated @Mon Jul 25 08:11:12 UTC 2022
Index Updated @Mon Jul 25 08:11:22 UTC 2022
Index Updated @Mon Jul 25 08:11:32 UTC 2022
Index Updated @Mon Jul 25 08:11

: 1

In [ ]:
podman run -d --name apache-httpd-index-updater --pod dynamic-webserver alpine sh -c 'while true; do echo "Index Updated @$(date)" >> /usr/local/apache2/htdocs/index.html; sleep 10; done'

## Cleanup Pods

Removing pods deletes also their running containers, which is a fast way to cleanup resources.

The command is pretty much the same as with containers, but prefixed with the word `pod`:

In [23]:
podman pod rm -f dynamic-webserver

22a922310d513a2f413dad1d765c3ef4b153f3ead7c2bb9d549e5693e3aa2b3f


: 1

## Advanced Pod Creation: The init pattern

The _init_ pattern is an advanced concept from Kuberbetes in which a container starts previous to other ones and performs some action (usuarlly set up or configure a dependent resource), so that other containers run properly afterwards.
This way each container has exactly one concern.


### Extending the previous example with an init container

Let's extend the pod example that we have seen with an _init_ container.

All in all we will have:

1. A pod with a _pause_ container which will be defined as _new_ when creating the first container, the _init_ container.
1. The _init_ container itself
1. The _main_ httpd container
1. The _sidecar_ (updater) container

#### Creating the pod with the init container

Note: It seems that using the init pattern we need to supply the volume explicitly on all subsequent commands.

In [24]:
podman create --name apache-httpd-init --init-ctr=always --pod new:dynamic-webserver -p 8080:80 --volume  webserver-data:/usr/local/apache2/htdocs:z busybox sh -c 'echo "Static Website Init" > /usr/local/apache2/htdocs/index.html'

3d4ffc9001b9355ce9de302ec333a1ab0398f88786da8807e055bc04934498b2


: 1

#### Creating the httpd server

Note: again we supply the volume in this command explicitly.

In [25]:
podman run -d --name apache-httpd --pod dynamic-webserver --volume  webserver-data:/usr/local/apache2/htdocs:z  httpd:2.4

c13b8d1cf103bbba4da70d2ddfe6247debd711ac3a73fe1f51dd947b5d11721f


: 1

#### Triggering the init container by starting the pod

We have provided the flag --init-ctr=always, so this way the pod will initialise each time that we start it.

To test this, let's stop and start the pod:

In [28]:
podman pod stop dynamic-webserver

80c71a708a8e3b7d702236aeff628e213c3fccee699fe17ce40510a6d99f815e


: 1

In [29]:
podman pod start dynamic-webserver

80c71a708a8e3b7d702236aeff628e213c3fccee699fe17ce40510a6d99f815e


: 1

In [30]:
curl localhost:8080

Static Website Init


: 1

#### Creating the sidecar container (updater)

Note: again we supply the volume in this command explicitly.

In [31]:
podman run -d --name apache-httpd-index-updater --pod dynamic-webserver --volume  webserver-data:/usr/local/apache2/htdocs:z  alpine sh -c 'while true; do echo "Index Updated @$(date)" >> /usr/local/apache2/htdocs/index.html; sleep 10; done'

b2adc1f961418019afaf4ceeb90fdc4da6509a134e038e7b6db52328d45ed2b2


: 1

In [32]:
curl localhost:8080

Static Website Init
Index Updated @Mon Jul 25 09:10:15 UTC 2022
Index Updated @Mon Jul 25 09:10:25 UTC 2022
Index Updated @Mon Jul 25 09:10:35 UTC 2022
Index Updated @Mon Jul 25 09:10:45 UTC 2022
Index Updated @Mon Jul 25 09:10:54 UTC 2022
Index Updated @Mon Jul 25 09:11:04 UTC 2022
Index Updated @Mon Jul 25 09:11:14 UTC 2022


: 1

In [33]:
podman pod stop dynamic-webserver

80c71a708a8e3b7d702236aeff628e213c3fccee699fe17ce40510a6d99f815e


: 1

In [36]:
curl localhost:8080

Static Website Init
Index Updated @Mon Jul 25 09:12:23 UTC 2022
Index Updated @Mon Jul 25 09:12:33 UTC 2022


: 1

## Generate a Kubernetes Pod Manifest from a Podman's Pod

Podman can generate Kubernetes manifests out of running Pods (and play manifests too!).


In [37]:
podman generate kube dynamic-webserver

# Save the output of this file and use kubectl create -f to import
# it into Kubernetes.
#
# Created with podman-4.1.1
apiVersion: v1
kind: Pod
metadata:
  annotations:
    io.kubernetes.cri-o.ContainerType/apache-httpd: container
    io.kubernetes.cri-o.ContainerType/apache-httpd-index-updater: container
    io.kubernetes.cri-o.ContainerType/apache-httpd-init: container
    io.kubernetes.cri-o.SandboxID/apache-httpd: dynamic-webserver
    io.kubernetes.cri-o.SandboxID/apache-httpd-index-updater: dynamic-webserver
    io.kubernetes.cri-o.SandboxID/apache-httpd-init: dynamic-webserver
    io.kubernetes.cri-o.TTY/apache-httpd: "false"
    io.kubernetes.cri-o.TTY/apache-httpd-index-updater: "false"
    io.kubernetes.cri-o.TTY/apache-httpd-init: "false"
    io.podman.annotations.autoremove/apache-httpd: "FALSE"
    io.podman.annotations.autoremove/apache-httpd-index-updater: "FALSE"
    io.podman.annotations.autoremove/apache-httpd-init: "FALSE"
    io.podman.annotations.init/apache-httpd:

: 1

In [38]:
# Clean export to remove unnecessary metadata

podman generate kube dynamic-webserver | yq 'del(.metadata.annotations , .metadata.creationTimestamp , .status)' --yaml-output

apiVersion: v14l
kind: Pod
metadata:
  labels:
    app: dynamic-webserver
  name: dynamic-webserver
spec:
  containers:
    - image: docker.io/library/httpd:2.4
      name: apache-httpd
      ports:
        - containerPort: 80
          hostPort: 8080
      resources: {}
      securityContext:
        capabilities:
          drop:
            - CAP_MKNOD
            - CAP_NET_RAW
            - CAP_AUDIT_WRITE
      volumeMounts:
        - mountPath: /usr/local/apache2/htdocs
          name: webserver-data-pvc
    - command:
        - sh
        - -c
        - while true; do echo "Index Updated @$(date)" >> /usr/local/apache2/htdocs/index.html;
          sleep 10; done
      image: docker.io/library/alpine:latest
      name: apache-httpd-index-updater
      resources: {}
      securityContext:
        capabilities:
          drop:
            - CAP_MKNOD
            - CAP_NET_RAW
            - CAP_AUDIT_WRITE
      volumeMounts:
        - mountPath: /usr/local/apache2/htdocs
          n

: 1